In [131]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

## Implementation of ASVD algorithm

In [145]:
def ASVD_reccommender(train_global_rating_mean, user, item, train_record, test_record, train_rating, test_rating, alpha):
    ASVD_u = train_global_rating_mean
    ASVD_bu = tf.Variable(tf.zeros((user, 1), dtype=tf.float32))
    ASVD_bi = tf.Variable(tf.zeros((1, item), dtype=tf.float32))
    ASVD_Q = tf.Variable(tf.random.normal([item, f], stddev=1/(f**0.5), dtype=tf.float32))
    ASVD_x = tf.Variable(tf.random.normal([item,f], stddev=1/(f**0.5), dtype=tf.float32))
    ASVD_y = tf.Variable(tf.random.normal([item,f], stddev=1/(f**0.5), dtype=tf.float32))
    ASVD_train_record = tf.constant(train_record, dtype=tf.float32)
    ASVD_test_record = tf.constant(test_record, dtype=tf.float32)
    N = np.zeros((user, item), dtype=np.float32)
    R = np.zeros((user, item), dtype=np.float32)
    ASVD_Nu = np.sum(train_record, axis=1)
    
    print(N.shape)
    print(user)
    print(ASVD_Nu.shape)

    for i in range(0, user):
        if ASVD_Nu[i] == 0:
            ASVD_Nu[i] = 1
    for i in range(0, item):
        N[:, i] = ASVD_Nu

    ASVD_Nu = N ** 0.5
    ASVD_Ru = np.sum(test_record, axis=1)

    for i in range(user):
        if ASVD_Ru[i] == 0:
            ASVD_Ru[i] = 1
    for i in range(item):
        R[:,i] = ASVD_Ru

    ASVD_Ru = R ** 0.5
    ASVD_QX = tf.matmul(ASVD_Q, ASVD_x, transpose_b=True)
    ASVD_Qxt = tf.matmul(ASVD_test_record, ASVD_QX)
    ASVD_Qxt =(test_rating - ASVD_u - ASVD_bi - ASVD_bu) * ASVD_Qxt
    ASVD_QY = tf.matmul(ASVD_Q, ASVD_y, transpose_b=True)
    ASVD_Qyt = tf.matmul(ASVD_train_record, ASVD_QY)
    ASVD_predict_rating = ASVD_u + ASVD_bu + ASVD_bi + ASVD_Qxt/ASVD_Ru + ASVD_Qyt / ASVD_Nu
    ASVD_loss = tf.reduce_sum(((train_rating-ASVD_predict_rating)*train_record)**2) + alpha * \
                             ((tf.reduce_sum(ASVD_x) ** 2) + (tf.reduce_sum(ASVD_y) ** 2) + \
                              (tf.reduce_sum(ASVD_bu) ** 2) + (tf.reduce_sum(ASVD_bi) ** 2) + \
                              (tf.reduce_sum(ASVD_Q) ** 2))
                             
    return ASVD_loss, ASVD_predict_rating

## Data structure research

In [146]:
articles = pd.read_csv('articles.csv')
customers = pd.read_csv('customers.csv')
transactions = pd.read_csv('transactions_train.csv')

In [147]:
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [148]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [149]:
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [150]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB


In [151]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [152]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ GB


## Data preprocessing

In [153]:
transactions.dropna(inplace=True)

In [154]:
customers['new_id'] = [i for i in range(customers.shape[0])]
transactions = transactions.merge(customers, on='customer_id', how='left')

In [155]:
transactions = transactions[['new_id', 'article_id']]
transactions['isbuy'] = 1
transactions

,new_id,article_id,isbuy
0,2,663713001,1
1,2,541518023,1
2,7,505221004,1
3,7,685687003,1
4,7,685687004,1
...,...,...,...
31788319,1371691,929511001,1
31788320,1371691,891322004,1
31788321,1371721,918325001,1
31788322,1371747,833459002,1


In [156]:
for_zeros = pd.DataFrame(customers['new_id'].loc[:80]).merge(pd.DataFrame(articles['article_id'].loc[:80]), how='cross')
for_zeros = pd.concat([for_zeros, transactions[['new_id', 'article_id']], \
                       transactions[['new_id', 'article_id']]]).drop_duplicates(keep=False)
for_zeros['isbuy'] = 0
transactions = pd.concat([transactions, for_zeros])

In [157]:
train_global_rating_mean = np.mean(transactions['isbuy'])
x, y = transactions[['new_id', 'article_id']], transactions[['isbuy']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)
train_rating = pd.merge(x_train, y_train, right_index=True, left_index=True)
test_rating = pd.merge(x_test, y_test, right_index=True, left_index=True)
user, item = pd.unique(transactions['new_id']).shape[0] + 1, pd.unique(transactions['article_id']).shape[0] + 1

In [158]:
f = 100
alpha = 0.01

In [159]:
train_record = train_rating > 0
train_record = np.array(train_record, dtype=int)
test_record = test_rating > 0
test_record = np.array(test_record, dtype=int)

In [160]:
loss, predict = ASVD_reccommender(train_global_rating_mean, user, item, train_record, test_record, \
                                  train_rating, test_rating, alpha)

MemoryError: Unable to allocate 531. GiB for an array with shape (1362282, 104548) and data type float32

In [144]:
learning_rate = 1e-4
optimizer = tf.optimizers.SGD(learning_rate)

In [ ]:
RMSE = (tf.reduce_sum(((test_rating - predict) * test_record)**2) / transactions.shape[0])**0.5
MAE = tf.reduce_sum((tf.abs(test_rating - predict) * test_record)) / transactions.shape[0]